## Features of the Dashboard:
### 1. Table of top restaurant chains
### 2. Bar chart of number of locations
### 3. Bar chart of average rating
### 4. Interactivity: Sort, hover, zoom

In [2]:
pip install dash pandas plotly


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import pandas as pd
import dash
from dash import dcc, html
import plotly.express as px

# Load the dataset
file_path = "Dataset .csv"
df = pd.read_csv(file_path, on_bad_lines='skip', quoting=1)

# Clean and preprocess
df['Aggregate rating'] = pd.to_numeric(df['Aggregate rating'], errors='coerce')
df['Votes'] = pd.to_numeric(df['Votes'], errors='coerce')

# Identify chains
chain_counts = df['Restaurant Name'].value_counts()
chains = chain_counts[chain_counts > 1]
df_chains = df[df['Restaurant Name'].isin(chains.index)]

# Group and summarize
chain_summary = df_chains.groupby('Restaurant Name').agg({
    'Aggregate rating': 'mean',
    'Votes': 'sum',
    'Restaurant Name': 'count'
}).rename(columns={
    'Aggregate rating': 'Average Rating',
    'Votes': 'Total Votes',
    'Restaurant Name': 'Number of Locations'
}).sort_values(by='Number of Locations', ascending=False).reset_index()

# Setup Dash app
app = dash.Dash(__name__)
app.title = "Restaurant Chains Dashboard"

# Layout
app.layout = html.Div([
    html.H1("Restaurant Chains Analysis", style={'textAlign': 'center'}),

    dcc.Graph(
        figure=px.bar(
            chain_summary.head(10),
            x='Restaurant Name',
            y='Number of Locations',
            title="Top 10 Restaurant Chains by Number of Locations",
            color='Number of Locations',
            color_continuous_scale='Blues'
        )
    ),

    dcc.Graph(
        figure=px.bar(
            chain_summary[chain_summary['Number of Locations'] >= 3].sort_values(by='Average Rating', ascending=False).head(10),
            x='Restaurant Name',
            y='Average Rating',
            title="Top 10 Chains by Average Rating (Min 3 Locations)",
            color='Average Rating',
            color_continuous_scale='Greens',
            range_y=[0, 5]
        )
    ),

    dcc.Graph(
        figure=px.scatter(
            chain_summary,
            x='Number of Locations',
            y='Average Rating',
            size='Total Votes',
            hover_name='Restaurant Name',
            title="Restaurant Chains: Locations vs Rating (Bubble Size = Popularity)",
            color='Average Rating',
            color_continuous_scale='Viridis',
            range_y=[0, 5]
        )
    )
])

if __name__ == '__main__':
    app.run(debug=True)
